# Алгоритм Франк-Вульфа (20 баллов)

In [3]:
import numpy as np
import math
import itertools
from matplotlib import pyplot as plt
import time
import warnings
warnings.filterwarnings("ignore")

Вспомним основные свойства логистической регрессии с кросс-энтропийной функцией потерь:
1. Модель:

$$
g(x, a_i) = \frac{1}{1 + \exp \left[- \langle x, a_i \rangle\right]};
$$

2. Функция потерь:

$$
\ell(g(x, a_i), b_i) = -b_i \log \left[g(x, a_i)\right] - (1 - b_i) \log \left[1 -  g(x, a_i)\right].
$$

3. Полный вид оптимизационной задачи:

$$
\min_{x \in \mathbb{R}^d} \left[ f(x) = \frac{1}{n} \sum_{i=1}^n \ell \left(g(x, a_i), b_i \right) + \frac{\lambda}{2} \| x \|^2_2 \right],
$$

4. Градиент функции потерь:

$$
\nabla f(x) = \frac{1}{n} \sum_{i=1}^n (g(x, a_i) - b_i) a_i + \lambda x.
$$
   
5. Константа $L$-гладкости оценивается как

$$
L \geq \frac{1}{4n} \lambda_{\max} \left[A^\top A \right] + \lambda,
$$

6. Константа $\mu$-сильной выпуклости оценивается как $\mu \leq \lambda$.

В качестве дата-матрицы $A$ и целевого вектора $b$ рассмотрим данные из датасета [_mushrooms_](https://github.com/BRAIn-Lab-teaching/OPTIMIZATION-METHODS-COURSE/blob/ПМИ_осень_2025/Datasets/mushrooms.txt). Ниже представлена функция загрузки датасета.

In [4]:
url = "https://raw.githubusercontent.com/BRAIn-Lab-teaching/OPTIMIZATION-METHODS-COURSE/%D0%9F%D0%9C%D0%98_%D0%BE%D1%81%D0%B5%D0%BD%D1%8C_2025/Datasets/mushrooms.txt"
!wget -O mushrooms.txt "$url"

--2025-11-17 17:47:27--  https://raw.githubusercontent.com/BRAIn-Lab-teaching/OPTIMIZATION-METHODS-COURSE/%D0%9F%D0%9C%D0%98_%D0%BE%D1%81%D0%B5%D0%BD%D1%8C_2025/Datasets/mushrooms.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 879712 (859K) [text/plain]
Saving to: ‘mushrooms.txt’

mushrooms.txt       100%[===================>] 859.09K  --.-KB/s    in 0.03s   

2025-11-17 17:47:27 (25.8 MB/s) - ‘mushrooms.txt’ saved [879712/879712]



In [5]:
from sklearn.datasets import load_svmlight_file

#файл должен лежать в той же директории, что и notebook
dataset = "mushrooms.txt"

data = load_svmlight_file(dataset)
A, b = data[0].toarray(), data[1]

# Необходимое линейное преобразование
b = b - 1

С помощью функции `train_test_split` разделите датасет в отношении 4 к 1 (обучающая выборка должна быть в 4 раза больше, чем тестовая). Поставьте параметр `random_state = 57`. В дальнейшем мы будем валидировать процесс обучения на тестовой выборке.

In [6]:
import sklearn
A_train, A_test, b_train, b_test = sklearn.model_selection.train_test_split(A, b, test_size=0.2, random_state = 57)

Для обучающей части $A_{train}$, $b_{train}$ оцените константы $L$ и $\mu$, положив равенство в полученной ранее оценке. Задайте $\lambda$ так, чтобы $\lambda \approx L / 1000$.

In [7]:
eigvals = np.linalg.eigvalsh(A_train.T.dot(A_train))
lambda_max = np.max(eigvals)
n_train = A_train.shape[0]
L = eigvals.max() / (4 * n_train)
mu = L / 1000
lambda_value = mu


assert math.isclose(L, 2.586914976545057,  rel_tol=1e-6),  "Константа L-гладкости найдена неверно"
assert math.isclose(mu, 0.002586914976545057, rel_tol=1e-6),  "Константа регуляризации найдена неверно"

Дополните функции подсчета сигмоиды, кросс-энтропии и градиента оптимизируемой функции.

In [8]:
def sigmoid(x):
    """
    Вычисляет сигмоидную функцию.

    Параметры:
        x (np.array): Входное значение

    Возвращает:
        sigmoid (np.array) Значение сигмоидной функции для входных данных
    """
    """
    Было предупреждение о переполнении экспоненты, пришлось придумывать костыли, чтобы не было переполнения
    """

    pos_mask = (x >= 0)
    neg_mask = (x < 0)

    z = np.zeros_like(x)
    z[pos_mask] = np.exp(-x[pos_mask])
    z[neg_mask] = np.exp(x[neg_mask])

    result = np.zeros_like(x)
    result[pos_mask] = 1 / (1 + z[pos_mask])
    result[neg_mask] = z[neg_mask] / (1 + z[neg_mask])

    return result


def loss(x, A=A_train, b=b_train, lambda_value=L/1000):
    """
    Вычисляет значение эмпирического риска.

    Параметры:
        x (np.array): Вектор параметров модели
        A (np.array): Матрица признаков обучающей выборки
        b (np.array): Вектор меток обучающей выборки
        lambda_value (float): Параметр регуляризации

    Возвращает:
        loss (float): Значение функции потерь
    """

    epsilon = 1e-10

    g_value = sigmoid(A @ x)

    g_value = np.clip(g_value, epsilon, 1 - epsilon)

    cross_entropy_loss = -np.mean(b * np.log(g_value) + (1 - b) * np.log(1 - g_value))

    regularization = (lambda_value / 2) * np.sum(x**2)

    loss = cross_entropy_loss + regularization

    return loss



def grad(x, A=A_train, b=b_train, lambda_value=L/1000):
    """
    Вычисляет градиент функции потерь.

    Параметры:
        x (np.array): Вектор параметров модели
        A (np.array): Матрица признаков обучающей выборки
        b (np.array): Вектор меток обучающей выборки
        lambda_value (float): Параметр регуляризации

    Возвращает:
        grad (np.array): Градиент функции потерь
    """

    predictions = sigmoid(A @ x)

    grad = (A.T @ (predictions - b)) / len(b) + lambda_value * x

    return grad


## Основная часть (10 баллов)

__Задача 1.__ В методе Франк-Вульфа мы не используем проекции на множество, а отрешиваем подзадачу для выполнения шага алгоритма. Данная подзадача называется __LMO (Linear Minimization Oracle)__, и для некоторого вектора $c$ и выпуклого замкнутого множества $\mathcal{X}$ принимает вид:

$$
\text{LMO}(c) = \arg \min_{s \in \mathcal{X}} \langle c, s \rangle.
$$

__а) (1 балл)__ Покажите, что для симплекса:

$$
\mathcal{X} = \left\{ s \in \mathbb{R}^d \mid s \succeq 0,~ \boldsymbol{1}^\top s = R \right\}
$$

решение LMO:

$$
s^* = R e_i,~ \text{где}~ i = \arg \min_{j = \overline{1, d}} c_j.
$$

Реализуйте `lmo_simplex`.

In [9]:
def lmo_simplex(g, R):
    """
    LMO для симплекса.

    Параметры:
        g (np.array): Градиент в текущей точке
        R (float): Размер симплекса

    Возвращает:
        s (np.array): LMO
    """

    i = int(np.argmin(g))
    s = np.zeros_like(g, dtype=float)
    s[i] = float(R)
    return s

__б) (1 балл)__ Покажите, что для $\ell_1$-шара:

$$
\mathcal{X} = \left\{ s \in \mathbb{R}^d \mid \|s\|_1 \leq R \right\}
$$

решение LMO:

$$
s^* = -R \text{sign}(c_i) e_i,~ \text{где}~ i = \arg \max_{j = \overline{1, d}} |c_j|.
$$

Реализуйте `lmo_l1_ball`.

In [10]:
def lmo_l1_ball(g, R):
    """
    LMO для L1-шара.

    Параметры:
        g (np.array): Градиент в текущей точке
        R (float): Размер симплекса

    Возвращает:
        s (np.array): LMO
    """

    i = int(np.argmax(np.abs(g)))

    s = np.zeros_like(g, dtype=float)
    s[i] = -R * np.sign(g[i])

    return s

__в) (2 балла)__ Реализуйте алгоритм Франк-Вульфа. На каждой итерации сохраняйте:

- Значение функции потерь;
- Норму градиента;
- Значение метрики

$$
\text{gap}(x^k) = \langle \nabla f(x^k), x^k - s^k \rangle;
$$

- Усредненное значение метрики $\text{gap}(x^k)$:

$$
\text{gap} = \frac{1}{k} \sum_{t = 0}^k \text{gap}(x^t).
$$

**Псевдокод алгоритма**

---

_Инициализация:_

- Размер шага $\left\{ \gamma_k = \frac{2}{k+2} \right\} _{k=0}$
- Начальная точка $ x^0 \in \mathbb{R}^d $
- Максимальное число итераций $K$

---

$k$_-ая итерация_:
   
1. Найти оптимальное направление:

$$
s^k = \arg \min_{s \in \mathcal{X}} \langle s, \nabla f(x^k) \rangle
$$

2. Обновить значение:

$$
x^{k + 1} = (1 - \gamma_k) x^k + \gamma_k s^k
$$

---

_Условие остановки:_
- Достигнуто максимальное число итераций $K$ или $\text{gap} (x^k) < \varepsilon$

---

_Выход:_
- Полученное значение $x^K$

In [ ]:
def frank_wolfe(A, b, lambda_value, grad, lmo, x_0, eps=1e-8, max_iter=1000, **params):
    """
    Алгоритм Франк-Вульфа.

    Параметры:
        A (np.array): Матрица признаков.
        b (np.array): Вектор целевых значений
        lambda_value (float): Параметр регуляризации
        grad (Callable): Функция вычисления градиента
        lmo (Callable): Функция вычисления LMO
        x_0 (np.array): Начальная точка
        eps (float): Точность сходимости
        max_iter (int): Максимальное количество итераций
        params : Именованные гиперпараметры метода
            params['gamma'](k) : шаг на k-ой итерации
            params['R'] : радиус множества

    Возвращает:
        x_k (np.array) : Найденное решение
        history (list) : История метрики и значения
    """
    x_k = x_0.copy()
    history = {
        'values': [],
        'norms': [],
        'gaps': [],
        'gaps_averaged': [],
        'times': []
    }
    gaps_sum = 0.0
    start_time = time.time()

    R = params['R']

    for k in range(max_iter):

        gamma = params['gamma'](k)

        g_k = grad(x_k, A, b, lambda_value)

        s_k = lmo(g_k, R)

        x_k = (1 - gamma) * x_k + gamma * x_k

        gap_k = np.dot(g_k, x_k - s_k)

        gaps_sum += gap_k

        # Сохранение истории
        history['values'].append(loss(x_k, A, b, lambda_value))
        history['norms'].append(np.linalg.norm(g_k))
        history['gaps'].append(gap_k)
        history['gaps_averaged'].append(gaps_sum / (k + 1))
        history['times'].append(time.time() - start_time)

        if history['gaps'][-1] < eps:
            break

    return x_k, history

Запустите оптимизацию на обучающей подвыборке для обоих типов LMO. В качестве значения радиуса для $L_1$-шара примите $R = 5$, размера симплекса $R = 1$, cтартовую точку возьмите в $0$.

Постройте сравнительный график сходимости: значение критерия сходимости $\text{gap} (x^k)$ и $\text{gap}$ от номера итерации.

In [ ]:
# Ваше решение (Code)

__г) (1 балл)__ Проанализируйте вектор $x^K$, полученный на выходе. Для этого сделайте гистограмму значений полученного вектора, а также гистограмму разреженности (отношения количества ненулевых компонент к нулевым). Выведите также топ-5 компонент вектора $x^K$ по модулю.

In [ ]:
# Ваше решение (Code)

Сделайте выводы.

In [ ]:
# Ваше решение (Markdown)

__д) (2 балла)__ Исследуйте зависимость значения метрики `accuracy` модели от радиуса $R$ для LMO на $L_1$-шаре. Рассмотрите следующие значения: $R = 5, 10, 20, 50, 100, 1000$.

Постройте 4 сравнительных графика:

- `accuracy` итоговой модели от значения $R$ (train/test),
- `accuracy` итоговой модели от количества ненулевых компонент решения (train/test),
- Количество ненулевых компоненты решения от $R$,
- $L_1$-норма решения от $R$.

In [ ]:
# Ваше решение (Code)

Проанализируйте зависимости, объясните полученные результаты.

In [ ]:
# Ваше решение (Markdown)

__Задача 2.__ Сравним теперь метод Франк-Вульфа с градиентным спуском с проекцией.

__а) (1 балл)__ Покажите, что для $\ell_1$-шара:

$$
\mathcal{X} = \left\{ y \in \mathbb{R}^d \mid \|y\|_1 \leq R \right\}
$$

решение проекции:

$$
\Pi_{\mathcal{X}}(x) =
\begin{cases}
    x, & \|s\|_1 \leq R, \\
    \text{sign}(x_i)(|x_i| - \lambda)_{+}~ \forall i \in \overline{1, d}, & \|s\|_1 > R,
\end{cases}
$$

где $\lambda$ определяется из уравнения

$$
\sum_{i = 1}^d (|x_i| - \lambda)_{+} = R.
$$

In [ ]:
# Ваше решение (Markdown)

Реализуйте `project_l1_ball`.

In [ ]:
def project_l1_ball(x, R):
    """
    Проекция вектора x на L1-шар радиуса R.

    Параметры:
        x (np.array): Точка, из которой считается проекция
        R (float): Размер симплекса

    Возвращает:
        y (np.array): Проекция
    """

    # YOUR CODE HERE

    return y

__б) (2 балла)__ Решите задачу оптимизации на обучающей выборке с помощью градиентного спуска с проекцией.

In [ ]:
def projected_gradient_descent(A, b, lambda_value, grad, proj, x_0, eps=1e-8, max_iter=1000, **params):
    """
    Градиентный спуск с проекцией.

    Параметры:
        A (np.array): Матрица признаков.
        b (np.array): Вектор целевых значений
        lambda_value (float): Параметр регуляризации
        grad (Callable): Функция вычисления градиента
        proj (Callable): Функция вычисления проекции
        x_0 (np.array): Начальная точка
        eps (float): Точность сходимости
        max_iter (int): Максимальное количество итераций
        params : Именованные гиперпараметры метода
            params['gamma'](k) : шаг на k-ой итерации
            params['R'] : радиус множества

    Возвращает:
        x_k (np.array) : Найденное решение
        history (list) : История метрики и значения
    """
    x_k = x_0.copy()

    history = {
        'values': [],
        'norms': [],
        'times': []
    }
    start_time = time.time()

    for k in range(max_iter):

        # YOUR CODE HERE

        # История
        history['values'].append(loss(x_k, A, b))
        history['norms'].append(np.linalg.norm(g_k))
        history['times'].append(time.time() - start_time)

    return x_k, history

Сравните градиентный спуск с проекцией и алгоритм Франк—Вульфа для $R = 5$.

In [ ]:
# Ваше решение (Code)

Постройте 3 сравнительных графика:
- Значение функции от номера итерации,
- Значение функции от времени,
- Значение $L_2$ нормы от итерации.

In [ ]:
# Ваше решение (Code)

Убедитесь в том, что метод Франка—Вульфа сходится быстрее, чем градиентный спуск с проекцией. Cравните число ненулевых компонент в итоговом решении.

In [ ]:
# Ваше решение (Code)

## Дополнительная часть (10 баллов)

In [ ]:
from tqdm import tqdm
import torch
from torch import optim
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18

__Задача 3.__ В этой задаче вам будет предложено сначала найти $\text{LMO}$ на ядерный шар заданого радиуса $R$. А затем реализовать алгоритм Франк—Вульфа для обучения нейронной сети. Такой оптимизатор является SOTA оптимизатором для пре-трейна Muon. Почитать про теоретическое обоснование можно [здесь](https://arxiv.org/pdf/2506.04192).

__а) (3 балла)__ Ядерной нормой матрицы $X \in \mathbb{R}^{n \times d}$ называется:

$$
\| X \|_* = \sum_{i = 1}^{\min \{n, d\}} \sigma_i,
$$

где $\sigma_i$ — сингулярные числа матрицы $X$. Покажите, что для ядерного шара:

$$
\mathcal{X} = \left\{ S \in \mathbb{R}^{n \times d} \mid \| S \|_* \leq R \right\}.
$$

решение LMO:

$$
S^* = -R u_1 v_1^\top,
$$

где $u_1$, $v_1$ — левый и правый сингулярные векторы матрицы $C$ при максимальном сингулярном числе.

_Указание: прочитайте про нормы Шаттена._

In [ ]:
# Ваше решение (Markdown)

__б) (1 балл)__  Допишите `LMO` для класса `NuclearNormBall`.

In [ ]:
class NuclearNormBall:
    """
    Ограничения — ядерный шар.
    """
    def __init__(self, l2_diameter):
        self._R = l2_diameter / 2

    @torch.no_grad()
    def SVD_power_iteration(self, W, iterations=20):
        """
        Power iteration для вычисления первых сингулярных векторов
        """
        n, m = W.shape
        u = F.normalize(W.new_empty(n).normal_(0., 1.), dim=0)
        v = F.normalize(W.new_empty(m).normal_(0., 1.), dim=0)
        for _ in range(iterations):
            v = F.normalize(torch.mv(W.t(), u), dim=0)
            u = F.normalize(torch.mv(W, v), dim=0)
        sigma = torch.dot(u, torch.mv(W, v))
        return u, sigma, v

    @torch.no_grad()
    def lmo(self, x):
        """
        LMO.

        Параметры:
            x (torch.Tensor): Градиент в текущей точке

        Возвращает:
            lmo_solution (torch.Tensor): LMO
        """

        # YOUR CODE HERE

        return lmo_solution

    @torch.no_grad()
    def shift_inside(self, x):
        """
        Масштабирование.
        """
        u, sigma, v = self.SVD_power_iteration(x.flatten(1))
        nuclear_norm_val = sigma.sum()
        if nuclear_norm_val > self._R:
            sigma_new = torch.diag(sigma * (self._R / nuclear_norm_val))
            return u @ sigma_new @ v
        else:
            return x

__в) (1 балл)__  Допишите `LMO` для класса `L1Ball`.

In [ ]:
class L1Ball():
    """
    Ограничения — L1 шар.
    """
    def __init__(self, l2_diameter):
        self._R = l2_diameter / 2

    @torch.no_grad()
    def lmo(self, x):
        """
        LMO.

        Параметры:
            x (torch.Tensor): Градиент в текущей точке

        Возвращает:
            lmo_solution (torch.Tensor): LMO
        """

        # YOUR CODE HERE

        return lmo_solution

    @torch.no_grad()
    def shift_inside(self, x):
        """
        Масштабирование.
        """
        x_norm = torch.norm(x, p=1)
        return self._R * x.div(x_norm) if x_norm > self._R else x

Ниже представлены вспомогательные функции, которые потребуются для инициализации модели.

In [ ]:
@torch.no_grad()
def get_avg_init_norm(layer, param_type=None, repetitions=100):
    """
    Вычисляет среднюю норму инициализации слоя по умолчанию
    """
    output = 0
    for _ in range(repetitions):
        layer.reset_parameters()
        output += torch.norm(getattr(layer, param_type), p=2).item()
    return float(output) / repetitions

@torch.no_grad()
def get_model_init_norms(moduleList):
    """
    Вычисляет среднюю норму инициализации всех слоев по умолчанию
    """
    init_norms = dict()
    for module, param_type in moduleList:
        if hasattr(module, 'reset_parameters'):
            param = getattr(module, param_type)
            avg_norm = get_avg_init_norm(module, param_type=param_type)
            init_norms[param.shape] = avg_norm
    return init_norms

@torch.no_grad()
def set_constraints(moduleList, setLMO=L1Ball, value=300):
    """
    Создаются ограничения L_1 шара для каждого слоя
    """
    constraintList = []
    init_norms = get_model_init_norms(moduleList)
    for module, param_type in moduleList:
        param = getattr(module, param_type)
        diameter = 2.0 * value * init_norms[param.shape]
        constraint = setLMO(diameter)
        constraintList.append((constraint, param))
    return constraintList

@torch.no_grad()
def make_feasible(constraintList):
    """
    Масштабирование.
    """
    for constraint, param in constraintList:
        feasible = constraint.shift_inside(param)
        param.copy_(feasible)

__г) (2 балла)__ Реализуйте оптимизатор `SFW`.

In [ ]:
class SFW(optim.Optimizer):
    """
    Реализация стохастического Франк—Вульфа

    Параметры:
        params (Iterable): Итерируемый объект параметров для оптимизации или словарь
        lr (float): Скорость обучения
        momentum (float): Коэффициент моментума
    """

    def __init__(self, params, lr=0.9, momentum=0.8):

        defaults = dict(lr=lr, momentum=momentum)
        super(SFW, self).__init__(params, defaults)

    @torch.no_grad()
    def step(self, closure=None):
        """
        Выполняет один шаг оптимизатора.

        Параметры:
            closure (Сallable): Замыкание, которое пересчитывает модель и возвращает loss

        Возвращает:
            loss (float): Значение функции потерь
        """

        loss = closure() if closure is not None else None

        for group in self.param_groups:
            lr = group['lr']
            momentum = group['momentum']
            constraint = group['constraint']

            for p in group['params']:
                if p.grad is None:
                    continue

                # YOUR CODE HERE

        return loss

__д) (3 балла)__ Запустите обучение ResNet18 на CIFAR10. Подберите гиперпараметры. Радиус для $L_1$ и ядерного шаров должен быть не больше 50.

In [ ]:
class CIFAR10(Dataset):
    def __init__(self, root='.', download=True):
        transform_train = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])

        transform_test = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])

        self.train_dataset = torchvision.datasets.CIFAR10(
            root = root,
            train = True,
            transform = transform_train,
            download = download,
        )
        self.test_dataset = torchvision.datasets.CIFAR10(
            root = root,
            train = False,
            transform = transform_test,
            download = download,
        )

In [ ]:
CIFAR10_dataset = CIFAR10(root='.')

In [ ]:
def trainer(num_epochs, batch_size, model_class, criterion, optimizer_class=SFW, optimizer_params=None,
            constraintList=None, dataset=None, device='cuda' if torch.cuda.is_available() else 'cpu'
           ):
    """
    Универсальная функция для обучения моделей PyTorch.

    Параметры:
        num_epochs (int): Количество эпох обучения
        batch_size (int): Размер батча для DataLoader
        model_class (nn.Module): Класс модели
        criterion (nn.Module): Функция потерь
        optimizer_class (optim.Optimizer): Класс оптимизатора
        optimizer_params (dict): Параметры оптимизатора
        scheduler_class (optim.lr_scheduler): Класс планировщика скорости обучения
        constraintList (list): Список ограничений
        dataset (Dataset): Объект датасета
        device (str): Устройство для вычислений

    Возвращает:
        model (nn.Module): Обученная модель
        metrics (dict): Словарь с логами
    """

    # Создаем загрузчики данных
    train_loader = DataLoader(dataset.train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(dataset.test_dataset, batch_size=batch_size, shuffle=False)

    model = model_class.to(device)

    make_feasible(constraintList)
    param_groups = []
    for constraint, param_list in constraintList:
        param_group = {'params': param_list, 'constraint': constraint}
        if optimizer_params:
             param_group.update(optimizer_params)
        param_groups.append(param_group)
    optimizer = optimizer_class(param_groups)

    metrics = {
        "train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": [],
    }

    def train_epoch():
        model.train()

        # YOUR CODE HERE

        return train_loss, train_acc

    def test_epoch():
        model.eval()

        # YOUR CODE HERE

        return test_loss, test_acc

    for epoch in range(num_epochs):
        start = time.time()

        train_loss, train_acc = train_epoch()
        test_loss, test_acc = test_epoch()

        # Сохраняем метрики
        metrics["train_loss"].append(train_loss)
        metrics["train_acc"].append(train_acc)
        metrics["test_loss"].append(test_loss)
        metrics["test_acc"].append(test_acc)

        elapsed = time.time() - start

        print(f"Epoch {epoch+1}/{num_epochs} | "
              f"Train Loss: {train_loss:.4f}, Acc: {train_acc:.2f}% | "
              f"Test Loss: {test_loss:.4f}, Acc: {test_acc:.2f}% | "
              f"Time: {elapsed:.2f}s")
    return model, metrics

Наложение ограничений только на сверточные и линейные слои.

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
model_sfw_nuclear = resnet18(weights=None, num_classes=10)

moduleList_nuclear = []
for name, module in model_sfw_nuclear.named_modules():
    if isinstance(module, (nn.Conv2d, nn.Linear)):
        moduleList_nuclear.append((module, 'weight'))

constraints_nuclear = set_constraints(moduleList=moduleList_nuclear, value=40.0, setLMO=NuclearNormBall)

config_sfw_nuclear = {
    'num_epochs': ...,
    'batch_size': ...,
    'model_class': resnet18(weights=None, num_classes=10),
    'criterion': nn.CrossEntropyLoss(reduction='mean'),
    'optimizer_class': SFW,
    'optimizer_params': {
        'lr': ...,
        'momentum': ...,
    },
    'constraintList': constraints_nuclear,
    'dataset': CIFAR10_dataset,
    'device': device,
}

print("Обучение ResNet18 с помощью SFW на ядерном шаре")
trained_model_sfw_nuclear, metrics_sfw_nuclear = trainer(**config_sfw_nuclear)

In [ ]:
model_sfw_l1 = resnet18(weights=None, num_classes=10)

moduleList_l1 = []
for name, module in model_sfw_l1.named_modules():
    if isinstance(module, (nn.Conv2d, nn.Linear)):
        moduleList_l1.append((module, 'weight'))

constraints_l1 = set_constraints(moduleList=moduleList_l1, value=50.0, setLMO=L1Ball)

config_sfw_l1 = {
    'num_epochs': ...,
    'batch_size': ...,
    'model_class': model_sfw_l1,
    'criterion': nn.CrossEntropyLoss(reduction='mean'),
    'optimizer_class': SFW,
    'optimizer_params': {
        'lr': ...,
        'momentum': ...,
    },
    'constraintList': constraints_l1,
    'dataset': CIFAR10_dataset,
    'device': device,
}

print("Обучение ResNet18 с помощью SFW на L1 шаре")
trained_model_sfw_l1, metrics_sfw_l1 = trainer(**config_sfw_l1)